In [1]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
n = 5
op1 = tf.while_loop(lambda i: i < n, lambda i: i+1, [0]) 
op2 = tf.while_loop(lambda i, x: i < n, lambda i, x: (i+1, x), (0,0))

In [3]:
def cond(t1, t2, i):
    return tf.less(i, iters)

def f1(): return tf.add(t1, 1), tf.add(t2, 1)
def increment(t1, t2):
    return f1

def f2(): return t2, t1
def swap(t1, t2):
    return f2
        
def cond_loop(t1, t2, iters):
    def body(t1, t2, i):
        t1, t2 = tf.cond(tf.less(i+1, iters),
                        increment(t1, t2),
                        swap(t1, t2))

        return [t1, t2, tf.add(i, 1)]
    return tf.while_loop(lambda t1, t2, i: tf.less(i, iters), 
                         body, 
                         [t1, t2, 0])

t1 = tf.constant(1)
t2 = tf.constant(5)
iters = tf.constant(3)

with tf.Session() as sess:
    loop = cond_loop(t1, t2, iters)
    print(sess.run(loop))

[5, 1, 3]


In [4]:
# load data
iris = datasets.load_iris()
x_vals = np.array([[x[0], x[3]] for x in iris.data])
y_vals = np.array([1 if y == 0 else -1 for y in iris.target])

In [5]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=2, random_state=42)
for train_index, test_index in kf.split(x_vals, y_vals):
    X_train, y_train, X_test, y_test = x_vals[train_index], y_vals[train_index], x_vals[test_index], y_vals[test_index]

In [6]:
_X_len = X_train.shape[0]
_X_dim = X_train.shape[-1]
_y_dim = y_train.shape[-1]
_max_iter=10000; _kernel_type='linear'; _C=1.0; _epsilon=1e-3

In [7]:
def svm_weights(X_len: int, X_dim: int, y_dim: int, scope="svm"):
    """[setup svm variables weights]

    Arguments:
        X_len {[int]} -- [total number of X instance]
        X_dim {[int]} -- [dimension number of X]
        y_dim {[int]} -- [dimension number of y]

    Keyword Arguments:
        scope {str} -- [description] (default: {"svm"})
    """
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        W = tf.get_variable("W", [X_dim, y_dim], initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable("b", [y_dim], initializer=tf.contrib.layers.xavier_initializer())
        alphas = tf.random_uniform([X_len], dtype=tf.float32) * _C
    return W, b, alphas

In [8]:
_X = tf.placeholder(tf.float32, shape=[None, _X_dim])
_y = tf.placeholder(tf.float32, shape=[None, _y_dim])
_W, _b, _alphas = svm_weights(_X_len, _X_dim, _y_dim)

In [9]:
def examineExample(i2: int, numChanged: int):
    """[take training sample X_train[i2] as \alpha_2 to check \alpha_1]

    Arguments:
        i {int} -- [training sample index]
        numChanged {int} -- [number of training changes for _alphas]

    Returns:
        cnt {int} - [if i2 is improved by heursitic search pair of relevant i1]
    """
    return i2+1, 0

In [10]:
numChanged = 0; examineAll = True

def smo_loop(numChanged: tf.Tensor, examineAll: tf.Tensor):
    """[loop processing for SMO]

    Arguments:
        numChanged {tf.Tensor} -- [if alpha_1 or alpha_2 is changed during the iteration]
        examineAll {tf.Tensor} -- [if all training sample is processed during the iteration]

    Returns:
        [type] -- [description]
    """
    # loop I over all training examples
    _, train_changed = tf.while_loop(lambda i2, numChanged: i2 < _X_len, examineExample, (0, 0))
    # loop I over examples where alpha is not 0 & not _C
    def breaker_body(i2, numChanged):
        i2, numChanged = tf.cond(
            tf.logical_and(tf.not_equal(_alphas[i2], 0), tf.not_equal(_alphas[i2], _C)), 
            examineExample,
            lambda: (i2+1, 0))
        return i2, numChanged
    
    _, breaker_changed = tf.while_loop(lambda i2, numChanged: i2 < _X_len,
                                       breaker_body, 
                                       (0, 0))
    numChanged = numChanged + tf.cond(examineAll, lambda: train_changed, lambda: breaker_changed)
    # just check if loop smo is working or not
    numChanged = tf.Print(numChanged, [numChanged], message="numChanged:")
    examineAll = tf.cond(examineAll, lambda: False, lambda: tf.cond(tf.equal(numChanged, 0), lambda: True, lambda: examineAll))
    return numChanged, examineAll

op = tf.while_loop(lambda numChanged, examineAll: tf.logical_or(numChanged > 0, examineAll), smo_loop, (numChanged, examineAll))

In [11]:
numChanged

0

In [12]:
examineAll

True